In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***  
<h2 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  Digit Recognizer usign CNN 
    </h2>

In [ ]:
# importign the necessary libiraries 
import pandas as pd 
import numpy as np 
import plotly.express as px
import matplotlib.pylab as plt
from pandas.api.types import CategoricalDtype

  <h3 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
Load the Date and define the Strategy
    </h3>

In [ ]:
# load the data 
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
# see the shape of the data 
print(f" X_train shape: {train.shape}")
print(f" X_test shape: {test.shape}")

####  before getting hands dirty here is the problem
- we want to make a model to recognize the digit from pixels [ image ] so `classification problem`
- the data splitting must be at this 
     - `train data : to train the model on` 
     - `validation data :  to see the performance of the model and tune the model on` 
     - `test data : a blind hold data that the model never seen before to genelalize the model performance to launch`  

### Define the aim you want to get 
<img src="https://th.bing.com/th/id/OIP.Tae_yCyuXRBV6-TfU_hteQHaGc?w=236&h=205&c=7&r=0&o=5&dpr=1.25&pid=1.7" style="margin:auto;width:600px;height:500px" >

- using `dev_set and single evaluation metric` that is the development set and a single number in our case i will use the `accuracy` . This is the aim i want to hit right in the center 

<p style="color:#333;font-size:17px">Before setting the aim and prevent wasting time in tunning the model in the wrong data  </p>
<p> let's make sure that the following is correct:</p>
     <p> The `Train/Dev/Test` data are from the same distribution.</p>
     <p> The  ` Test` data are big enough to give more confidence.</p>
    
- Note: ` Data From the Samse distribution ` means many things, like not training the model on high reselution images and then test it on blury images and so on.

### I know that that is alot for such a simple problem but the strategy is all that matters when it comes to big problem

<h1 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  View the data and have insigts
    </h1>

In [ ]:
# have alook at the data 
train.head()

In [ ]:
labels = train["label"].value_counts()
labels 

In [ ]:
# see the labels 
fig=px.bar(x=labels.index , y=labels.values,title="Number of each Digit Images",
       labels={"x":"Digit","y":"Number of Images"},
       text=labels.values,
       color_discrete_sequence=["#17202A"],
      )
fig.update_layout(title_x=.5,title_font_color="#fff",title_font_size=20,paper_bgcolor="#333",font_color="#bbb",
                 plot_bgcolor="#ccc",
                   xaxis = dict(
                    tickmode = 'array',
                    tickvals  =list(range(0,10)),
                    ticktext= labels.index
                     )
                 )
fig.update_yaxes(showgrid=False)
fig.show()


#### searching for imbalance:
- what is `Imbalance` : case in wich some labels bigger than the others, so the model will tend to overfit these labels than others and this will case bigge losses in the other labels
- ` In our case i think the differnce will not make imbalance`, if you have other opinion please tell me.

  <h1 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  Splitting and Reshaping the data 
    </h1>

- we have the train
- we will split the train data into `train/dev` sets

In [ ]:
# take labels out of the date 
train_x = train.drop("label",axis=1)
train_y= train["label"]
train_x.head()

In [ ]:
# splitting the test into dev/blind sets
from sklearn.model_selection import train_test_split
train_x ,dev_x ,train_y, dev_y = train_test_split(train_x,train_y,test_size=.1,stratify=train_y)

In [ ]:
# show the data shapes
print(f" X_train shape: {train_x.shape}")
print(f" X_dev shape: {dev_x.shape}")
print(f" X_test shape: {test.shape}")

## this is an intial splitting of the model and then train it in the full Train data

### Reshaping the date:
- to feed the date into the CNN it must be an image , in our case only a (28,28,1) shape which is a simple gray image
- using numpy to reshape the date

In [ ]:
# frist conver the date frame into numpy array
train_x = train_x.to_numpy()
dev_x   = dev_x.to_numpy()
test    = test.to_numpy()


In [ ]:
# second : reshape the array
train_x = train_x.reshape(-1,28,28,1)
dev_x   = dev_x.reshape(-1,28,28,1)
test    = test.reshape(-1,28,28,1)

In [ ]:
# show the data shapes
print(f" X_train shape: {train_x.shape}")
print(f" X_dev shape: {dev_x.shape}")
print(f" X_test shape: {test.shape}")

  <h1 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  visualize the array into images
    </h1>

- in this section i will show some random images from the data

### showing some traingin date 

In [ ]:
# get some random index fro the train
random_index = np.random.choice(range(len(train_x)),12)
# show the 12 random images 
plt.figure(figsize=(15,15))
plt.title("Some Random Training Images")
for i in range(12):
    plt.subplot(3,4,i+1)
    plt.imshow(train_x[random_index[i]],cmap=plt.get_cmap("gray"))
    plt.axis("off")
plt.tight_layout()
plt.show() 

### showing some test images

In [ ]:
# get some random index fro the train
random_index = np.random.choice(range(len(test)),12)
# show the 12 random images 
plt.figure(figsize=(15,15))
plt.title("Some Random Training Images")
for i in range(12):
    plt.subplot(3,4,i+1)
    plt.imshow(train_x[random_index[i]],cmap=plt.get_cmap("gray"))
    plt.axis("off")
plt.tight_layout()
plt.show() 

<p>we clearly see that the train and test date have the same distributions so there will be no </p> `data mismatch problem`

  <h1 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  Building the model `CNN` 
    </h1>

- using the CNN network which we know better when dealing with imags and defining features in it 
- model structure:
   - resizing layer (wich well resize the image into 28,28   ` this is not neccesary `
   - rescaling layer ( rescale the image .1/255 )
   - convolutional layer  with 64 feature kernels and kernel size (3,3) and valid padding 
   - maxpooling layer ( to reduce the image size into half (14,14 )
   - convolutional layer with 32 feature kernels and kernel size (3,3) and valid padding 
   - maxpooling layer ( to reduce the image size into half  nearly( 7,7 )
   - convolutional layer with 20 feature kernels and kernel size (3,3) and valid padding 
   - `flatten` layer: this layer takes all the images and flatten them in orer to feed into the dense layer wich is the final layer
   - dense layer with `softmax` activation fucntion and 10 neurons `one for each digit`


#### but before getting into the model there is something to do 
- one_hot_encoding the label : the layer at the end of the CNN has 10 neurons one for each digit and target label must be of the same size to apply `the cost fuction and gradient desent` 
- for exmaple: if the label of one row is [ digit 0 ] the y must be 1 at the neuron that represent the [digit 0 ]

In [ ]:
# for that lsets use pandas.getdummies method
train_y = pd.pandas.get_dummies(train_y)
dev_y   = pd.pandas.get_dummies(dev_y)

In [ ]:
train_y.head() 

In [ ]:
dev_y.head() 

- we can see that pandas ordered the digits from [ 0 to 9 ]
- so, the order of the output of the last `softmax layer` gives the probalility of gettign the digits with the same order

In [ ]:
# import the dependencies
from keras.models import Sequential
from keras.layers import Flatten , Conv2D , Dense , MaxPool2D , BatchNormalization,Resizing,Rescaling,Dropout
from keras.callbacks import EarlyStopping , ModelCheckpoint
from keras import regularizers
import tensorflow as tf 

In [ ]:
# image size
img_size = 28
scale =1./255
batch_size=32 
input_shape = (28,28,1)
#building the model
model = Sequential()
model.add(Resizing(img_size,img_size))
model.add(Rescaling(scale))
model.add(Conv2D(10,kernel_size=3,activation="relu",padding="valid",input_shape=input_shape))
model.add(MaxPool2D(2))
model.add(Conv2D(20,kernel_size=3,activation="relu",padding="valid"))
model.add(MaxPool2D(2))
model.add(Conv2D(30,kernel_size=3,activation="relu",padding="same"))
model.add(Flatten())
model.add(Dense(10,activation="softmax")) 

In [ ]:
# now compiling the model
model.compile(optimizer="adam",metrics=["accuracy"],loss = "categorical_crossentropy")

In [ ]:
# fitting the model 
history = model.fit(train_x,train_y,validation_data=(dev_x,dev_y),epochs=30,batch_size=batch_size)

<h1 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  See the model history performance
    </h1>

In [ ]:
X = range(len(history.history["loss"]))
train_loss = history.history["loss"]
val_loss =history.history["val_loss"]
train_acc = history.history["accuracy"]
val_accuracy =history.history["val_accuracy"]

In [ ]:
# show the performance 
import seaborn as sns 
plt.style.use("seaborn")
plt.figure()
plt.title("Loss history")
sns.lineplot(x=X,y=train_loss,label="Train",marker="o")
sns.lineplot(x=X,y=val_loss,label="val",marker="o")
plt.show()
plt.figure()
plt.title("Accuracy history")
sns.lineplot(x=X,y=train_acc,label="Train",marker="o")
sns.lineplot(x=X,y=val_accuracy,label="val",marker="o")
plt.show()

# From the above perfromance there is something clear
- the model is `OVERFITTING` the train data 
## To sole these problem we can these methods:
- try a simpler model 
- getting more date ` which in our case not possible`
- add requlalization term it can be `L2` or `Dropout` 

### let's try a `dropout` regularization`, the same model but with additional dropout layers

In [ ]:
# image size
img_size = 28
scale =1./255
batch_size=32 
input_shape = (28,28,1)
#building the model
model_2 = Sequential()
model_2.add(Resizing(img_size,img_size))
model_2.add(Rescaling(scale))
model_2.add(Conv2D(10,kernel_size=3,activation="relu",padding="valid",input_shape=input_shape))
model_2.add(Dropout(.3))
model_2.add(MaxPool2D(2))
model_2.add(Conv2D(30,kernel_size=3,activation="relu",padding="valid"))
model_2.add(Dropout(.3))
model_2.add(MaxPool2D(2))
model_2.add(Conv2D(64,kernel_size=3,activation="relu",padding="same"))
model_2.add(Dropout(.5))
model_2.add(Flatten())
model_2.add(Dense(10,activation="softmax")) 

In [ ]:
#modelcheck to save model best weights 
best_model = ModelCheckpoint("best.h5",monitor="val_accuracy")

In [ ]:
# now compiling the model
model_2.compile(optimizer="adam",metrics=["accuracy"],loss = "categorical_crossentropy")
# fitting the model 
history_2 = model_2.fit(train_x,train_y,validation_data=(dev_x,dev_y),epochs=15,batch_size=batch_size,callbacks=[best_model])

In [ ]:
X = range(len(history_2.history["loss"]))
train_loss = history_2.history["loss"]
val_loss =history_2.history["val_loss"]
train_acc = history_2.history["accuracy"]
val_accuracy =history_2.history["val_accuracy"]

In [ ]:
plt.style.use("seaborn")
plt.figure()
plt.title("Loss history")
sns.lineplot(x=X,y=train_loss,label="Train",marker="o")
sns.lineplot(x=X,y=val_loss,label="val",marker="o")
plt.show()
plt.figure()
plt.title("Accuracy history")
sns.lineplot(x=X,y=train_acc,label="Train",marker="o")
sns.lineplot(x=X,y=val_accuracy,label="val",marker="o")
plt.show()

### Waht we can see:
- The model is much better than the prevoius one 
#### you may wonder why the validation accuracy is always bigger than the train accuracy ?
### ans: using the dropout lead to this in other words `When training, a percentage of the features are set to zero (50% in our case since we are using Dropout(0.5)). When testing, all features are used (and are scaled appropriately). So the model at test time is more robust - and can lead to higher testing accuracies`.

<h1 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  Model from experiments
    </h1>

- this model archeticure is taken from experiment from a great notebook <a href="https://www.kaggle.com/code/cdeotte/how-to-choose-cnn-architecture-mnist/notebook"> Here <a>
- this model have also Dropout and Batchnormalization layers beside to conv and dense layers

In [ ]:
# data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

model_l= Sequential()

model_l.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model_l.add(BatchNormalization())
model_l.add(Conv2D(32,kernel_size=3,activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Dropout(0.4))

model_l.add(Conv2D(64,kernel_size=3,activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Conv2D(64,kernel_size=3,activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Dropout(0.4))

model_l.add(Flatten())
model_l.add(Dense(128, activation='relu'))
model_l.add(BatchNormalization())
model_l.add(Dropout(0.4))
model_l.add(Dense(10, activation='softmax'))

model_l.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# now compiling the model
model_l.compile(optimizer="adam",metrics=["accuracy"],loss = "categorical_crossentropy")
# fitting the model 
lenet = model_l.fit_generator(datagen.flow(train_x,train_y,batch_size=64),validation_data=(dev_x,dev_y),epochs=50,callbacks=[best_model])

In [ ]:
X = range(len(lenet.history["loss"]))
train_loss = lenet.history["loss"]
val_loss =lenet.history["val_loss"]
train_acc = lenet.history["accuracy"]
val_accuracy =lenet.history["val_accuracy"]

In [ ]:
plt.style.use("seaborn")
plt.figure()
plt.title("Loss history")
sns.lineplot(x=X,y=train_loss,label="Train",marker="o")
sns.lineplot(x=X,y=val_loss,label="val",marker="o")
plt.show()
plt.figure()
plt.title("Accuracy history")
sns.lineplot(x=X,y=train_acc,label="Train",marker="o")
sns.lineplot(x=X,y=val_accuracy,label="val",marker="o")
plt.show()

In [ ]:
# get best wegits
model_l.load_weights("best.h5")

In [ ]:
# making the submession
sub = pd.DataFrame()
sub["ImageId"] = range(1,len(test)+1)
probalilites = model_l.predict(test)
digits =[0,1,2,3,4,5,6,7,8,9]
labels = [digits[np.argmax(prob)] for prob in probalilites]
sub["Label"] =labels
sub.set_index("ImageId")
sub.to_csv('submission.csv', index=False)

<h1 style="background-color:#333;color:#fff;padding:10px;text-align:center;border-radius:10px">
  visualize model performance
    </h1>

In [ ]:
# making a fuction that predict digit 
def prdict(image):
    image =image.reshape(1,28,28,1)
    labels = [0,1,2,3,4,5,6,7,8,9]
    probalility_prdicted = model_l.predict(image)
    predicted_digit = labels[np.argmax(probalility_prdicted)]
    return predicted_digit

In [ ]:
prdict(test[300])

In [ ]:
model.predict(test[0].reshape(1,28,28,1)).shape

In [ ]:
# get some random index fro the train
random_index = np.random.choice(range(len(test)),12)
# show the 12 random images 
plt.figure(figsize=(15,15))
for i in range(12):
    plt.subplot(3,4,i+1)
    predicted_digit = prdict(test[random_index[i]])
    plt.imshow(test[random_index[i]],cmap=plt.get_cmap("gray"))
    plt.title(f"Model Prediction: {predicted_digit}")
    plt.axis("off")
plt.tight_layout()
plt.show() 